# Classroom problems

## 1. Comparment-based approach to Diffusion

In [ ]:
using Distributions

In [ ]:
"""
    L - length in mm of the one-dimensional domain where molecules of chemical species A diffuse 
    K - number of compartments 
    h - length of a single compartment in μm (volume: h³)
    D - Diffusion constant in mm²sec⁻¹
    d - rate constant (d = D/h^2)
"""

L = 1            
K = 40
h = ####     
D = 0.0001
d = ####;

In [ ]:
gillespie_alg_pure_diffusion = function(A₀, Nt)

    """
    Inputs:
        A₀ - a K-dimensional vector with the initial amount of molecules insided each compartment. 
        Nₜ - maximum number of jumps between compartments.
    Outputs:
        tt - a partitioned time interval [0, T] with times sampled from an exponential distribution
        AA - the molecule count for each time t ∈ tt
    """
    
    t = 0.0

    tt = [t]
    
    AA = zeros(Nt, K)
    AA[1, :] = A₀

    
    for t_idx in 1:Nt-1
     
        # Step 1
        # Generate two random numbers r₁, r₂ (r₁ used to sample from the Exponential distribution, r₂ to sample the compartment)
        
        r₁, r₂ = ####

        # Step 2
        # Compute propensity functions
        α = ####
        α₀ = ####

        # Step 3
        # Compute the time at which the next chemical reaction takes place as t+τ, where τ is sampled from an Exponential distribution with intensity α₀
        τ = (1/α₀)*log(1/r₁)                                   
        t += τ

        # Step 4
        # Find j ∈ [1, 2, ..., K-1] for which one molecule diffuses to the right or j ∈ [2, 3, ..., K] for which one molecule diffuses to the left
        prob_right = sum(α[1:K-1]/α₀)
        if r₂ < prob_right
            index_j = find_j(1:K-1, α[1:K-1]/α₀, r₂)
            AA[t_idx+1, :] = copy(AA[t_idx, :])
            AA[t_idx+1, index_j] = ####AA[t_idx, index_j]-1
            AA[t_idx+1, index_j+1] = ####AA[t_idx, index_j+1]+1
        else
            cte = α₀*prob_right
            index_j = find_j(2:K,([α[2]+cte; α[3:end]])/α₀, r₂)
            AA[t_idx+1, :] = copy(AA[t_idx, :])
            AA[t_idx+1, index_j] = ####
            AA[t_idx+1, index_j-1] = ####
        end

        append!(tt, t)

    end    
    return tt, AA
end

In [ ]:
find_j = function(idx, ww, r)
    """
    Inputs:
        idx - indexing vector. 
        ww - vector of probabilities for each i ∈ idx.
        r - random value in the interval (0, 1)
    Output:
        a single sampled i, where i ∈ idx.
    """
    ww_cumsum = cumsum(ww)
    return idx[findall(a -> a > r, ww_cumsum)[1]]
end

In [ ]:
A₀ = ####

Nt = #### # maximum number of jumps between compartments.

tt, AA = gillespie_alg_pure_diffusion(A₀, Nt)

In [ ]:
using Plots
anim = @animate for t_idx in 1:500:Nt
    bar(0.5*h/1000:h/1000:L, AA[t_idx,:], colour=:darkgrey,
            title="t: ".*"$(round(tt[t_idx]/60, digits = 1))".*" minutes",
            label = "Compartment-based diffusion",
            xlabel = "x [mm]",
            ylabel = "number of molecules",
            ylims = (0,500)) # 2:end since end = 1, periodic condition
end
gif(anim)

## 2. Compartment-based approach to Reaction-Diffusion with Production and Degradation

$\mathcal{A}_1 \overset{d}{\underset{d}\rightleftarrows}\mathcal{A}_2\overset{d}{\underset{d}\rightleftarrows}\mathcal{A}_3\overset{d}{\underset{d}\rightleftarrows}\dots\overset{d}{\underset{d}\rightleftarrows}\mathcal{A}_K,$ 

$\mathcal{A}_i\xrightarrow{\kappa_1}\emptyset\quad$ for $i=1,2,\dots,K,$

$\emptyset \xrightarrow{\kappa_2}\mathcal{A}_i\quad$ for $i=1,2,\dots,K/5.$

In [ ]:
κ₁, κ₂ = ####)

prepatternfactor = [0, 1/5];

In [ ]:
propensities_degra_prod_diffusion = function(A, d, κ, prepatternfactor)
    κ₁, κ₂ = κ
    K = length(A)
    α = zeros(K)
    for i in 1:K
        α[i] = ####
    end
    return #####
end

In [ ]:
gillespie_alg_degra_prod_diff = function(A₀, Nt)
    
    """
    Inputs:
        A₀ - a K-dimensional vector with the initial amount of molecules inside each compartment. 
        Nₜ - the maximum number of jumps for one simulation.
    Outputs:
        tt - a partitioned time interval [0, T] with times sampled from an exponential distribution.
        AA - the history of counts, counts saved at each time t ∈ tt.
    """
    
    K = length(A₀)
    
    t = 0.0

    tt = [t]
    AA = Integer.(zeros(Nt, K))
    AA[1,: ] = A₀

    κ = [κ₁, κ₂]
    
    for t_idx in 1:Nt-1
        
        # Step 1
        # Generate two random numbers r₁, r₂ (r₁ used to sample from the Exponential distribution, r₂ to sample the compartment index)
        r₁, r₂ = ####
        
        # Step 2
        # Compute propensity functions
       
        α = propensities_degra_prod_diffusion(AA[t_idx,:], d, κ, prepatternfactor)
        α₀ = ####

        # Step 3
        # Compute the time at which the next chemical reaction takes place as t+τ, where τ is sampled from an Exponential distribution with intensity α₀
        τ = ####                                  
        t += τ

        # Step 4
        # Find j ∈ [1, 2, ..., K-1] for which one molecule diffuses to the right or j ∈ [2, 3, ..., K] for which one molecule diffuses to the left or
        # j ∈ [1, 2, ..., K] for which one molecule degrades or j ∈ [1, 2, ..., K/5] for which one molecule is produced.
        
        prob_right = sum(α[1:K-1]/α₀)
        prob_left = sum(α[2:K]/α₀)
        prob_degra = sum(α[(K+1):2K]/α₀)
        #prob_prod = sum(α[(2K+1):end]/α₀)

        #Diffusion to the right
        if r₂ < prob_right 
            index_j = find_j(1:K-1, α[1:K-1]/α₀, r₂)
            AA[t_idx+1, :] = copy(AA[t_idx, :])
            AA[t_idx+1, index_j] = ####
            AA[t_idx+1, index_j+1] = ####
        
        #Diffusion to the left
        elseif r₂ < prob_right + prob_left 
            cte = α₀*prob_right
            index_j = find_j(2:K, ([α[2]+cte; α[3:K]])/α₀, r₂)
            AA[t_idx+1, :] = copy(AA[t_idx, :])
            AA[t_idx+1, index_j] = ####
            AA[t_idx+1, index_j-1] = ####
        
        #Degradation of each volume
        elseif r₂ < prob_right + prob_left + prob_degra
            cte = α₀*(prob_right + prob_left)
            index_j = find_j(1:K, ([α[K+1]+cte; α[K+2:2K]])/α₀, r₂)
            AA[t_idx+1, :] = copy(AA[t_idx, :])
            AA[t_idx+1, index_j] = ####
        
        #Production at the first Kfactor volumes
        else 
            cte = α₀*(prob_right + prob_left + prob_degra)
            index_j = find_j(1:Integer(K*prepatternfactor[end]), [α[2K+1]+cte; α[(2K+2):end]]/α₀, r₂)
            AA[t_idx+1, :] = copy(AA[t_idx, :])
            AA[t_idx+1, index_j+1] += ####
        end
        append!(tt, t)

    end    
    return tt, AA
end

In [ ]:
A₀ = ####

Nt = 200000 # maximum number of jumps between compartments.

tt, AA = gillespie_alg_degra_prod_diff(A₀, Nt)

In [ ]:
using Plots
anim = @animate for t_idx in 1:500:Nt
    bar(h/1000:h/1000:L, AA[t_idx,:], colour=:darkgrey,
            title="t: ".*"$(round(tt[t_idx]/60, digits = 1))".*" minutes",
            label = "Compartment-based approach to reaction-diffusion",
            xlabel = "x [mm]",
            ylabel = "number of molecules",
            ylims = (0,150)) # 2:end since end = 1, periodic condition
end
gif(anim)

The French flag model 

In [ ]:
bound₁ = 80
bound₂ = 30;

In [ ]:
x = h/1000:h/1000:L
y = AA[end,:]

plot(x[y .> bound₁], y[y .> bound₁]; 
    seriestype=:bar, colour=:darkblue, label = "N > 80")
plot!(x[bound₁ .>= y .> bound₂], y[bound₁ .>= y .> bound₂]; 
    seriestype=:bar, colour=:white, label = "80 >= N > 30")
plot!(x[y .<= bound₂], y[y .<= bound₂]; 
    seriestype=:bar, colour=:red, label = "30 >= N")
plot!(x, bound₁*ones(length(x)), linestyle=:dash, colour=:black, label=false)
plot!(x, bound₂*ones(length(x)), linestyle=:dash, colour=:black, label=false)

## 3. PDE solution

In [ ]:
xx = range(.0, stop = L, length = K+1)             # bin limits
Δx = xx[2]-xx[1]
xx_centre = xx[1:end-1] .+ Δx/2                    # bin centroids

A₀ = zeros(K)
p = [κ₁, κ₂]

In [ ]:
#import Pkg; Pkg.add(["DiffEqOperators", "DifferentialEquations"])

In [ ]:
using DiffEqOperators
# Second order approximation to the second derivative
order = 2
deriv = 2
∂² = CenteredDifference(deriv, order, h/1000, K)
nc = Neumann0BC(h/1000, 1)

In [ ]:
function discretized_pde!(du,u,p,t)
    du .= ####
end

In [ ]:
using DifferentialEquations

T = tt[end]
sol = solve(ODEProblem(discretized_pde!, A₀, (0, T)), Tsit5())

Plotting

In [ ]:
using Plots

x = (h/1000)/2:h/1000:L
y = AA[end,:]

p₁ = plot(x[y .> bound₁], y[y .> bound₁],
    ylims=(0,150),
    seriestype=:bar, colour=:darkblue, label = "N > 80",
    xlabel = "x [mm]", ylabel = "number of molecules",
    title = "compartment-based")
    plot!(x[bound₁ .>= y .> bound₂], y[bound₁ .>= y .> bound₂]; 
    seriestype=:bar, colour=:white, label = "80 >= N > 30")
    plot!(x[y .<= bound₂], y[y .<= bound₂]; 
    seriestype=:bar, colour=:red, label = "30 >= N")
    plot!(x, bound₁*ones(length(x)), linestyle=:dash, colour=:black, label=false)
    plot!(x, bound₂*ones(length(x)), linestyle=:dash, colour=:black, label=false)


y = sol(tt[end])*h^3

yblue = y.*(y.>bound₁)
ywhite = y.*(bound₂.>y.>=bound₁)
yred = y.*(y.<=bound₂)

p₂ = plot(x, y, fillrange=y.-yblue, 
    ylims=(0,150),
    linetype=:steppre, colour = :black, fill=:darkblue,  label = "N > 80",
    xlabel = "x [mm]", ylabel = "number of molecules",
    title = "PDE solution")
    plot!(x, y, fillrange=y.-ywhite, 
    linetype=:steppre, colour = :black,fill=:white,  label = "80 >= N > 30")
    plot!(x, y, fillrange=y.-yred, 
    linetype=:steppre, colour = :black,　fill=:red,　label = "30 >= N")
    plot!(x, bound₁*ones(length(x)), linestyle=:dash, colour=:black, label=false)
    plot!(x, bound₂*ones(length(x)), linestyle=:dash, colour=:black, label=false)

layout = @layout [p₁ p₂]
Plots.plot(p₁, p₂; layout = layout,  colorbar = false, size = (1400, 800),  margin = 30Plots.px)

## 4. Velocity-jump process

In [ ]:
gillespie_alg_degra_prod_velocity = function(Δt, T)

        
    """
    Inputs:
        Δt - length of time step. 
        T - stopping time.
    Outputs:
        tt - a partitioned time interval [0, T] with times sampled from an exponential distribution.
        xx - the history of counts, counts saved at each time t ∈ tt.
    """
    
    
    Nt = Integer(round(T/Δt))

    t = 0.0

    tt = [t]

    κ = [κ₁, κ₂]
    ensemble = [[x₀], [v₀]]

    for t_idx in 1:Nt-1
      
        xx = ensemble[1][]
        vv = ensemble[2][]
        
        # Generate a random numer r ∼ U(0,1) 
        # Assume that a particle moves along the x-axis at a constant speed s and compute the position of the molecule at the future time t+Δt 
        # Remember to apply reflective boundaries
        
        for i in 1:length(xx)
            xx[i] =  abs.(xx[i]+vv[i]*Δt)
            if xx[i]>L
                xx[i] = ####
            end
            r = rand()
            # check if the particle turns in the time interval t+Δt
            if r<λ*Δt
                vv[i] = ####
            end
        end

        # for each molecule, check whether they have to be removed from the system
        id_to_remove = []
        for i in 1:length(xx)
            r₁ = rand()
            if r₁ < κ₁*Δt 
                append!(id_to_remove, i)
            end
        end

        valid_id = setdiff(collect(1:length(xx)), id_to_remove)
        xx = xx[valid_id]
        vv = vv[valid_id]

        # check whether a new molecule should be introduced to the system
        r₂ = rand()
        if r₂ < (κ₂*(h)^2*L*1000/5)*Δt
            r₃ = rand()
            xx = [xx; ####] 
            vv = [vv; v₀] 
        end

        ensemble = [[xx], [vv]]
        t = t+Δt
        append!(tt, t)
    end    
    return tt, ensemble[1]
end

In [ ]:
s = 10^(-2)
λ = s^2/(2D);

Δt = 0.01
x₀ = [0.0]
v₀ = [s]

tt, xx = gillespie_alg_degra_prod_velocity(Δt, T)

Plotting

In [ ]:
#import Pkg; Pkg.add("StatsBase")
using StatsBase

In [ ]:
#x = h/1000:h/1000:L
x = (h/1000)/2:h/1000:L
y = copy(xx[end])


firstEdge = 0.0
lastEdge = L
binSize = h/1000
EdgeRange = (firstEdge:binSize:lastEdge)

histogram_fit = fit(Histogram, y, h/1000:h/1000:L)
weights =  histogram_fit.weights
edges = histogram_fit.edges[1]

idx = findall(>(0), weights .> bound₁ )
xblue = edges[idx]
yblue = weights[idx]

idx = findall(>(0), bound₁  .>= weights .> bound₂)
xwhite = edges[idx]
ywhite = weights[idx]

idx = findall(>(0), weights .<= bound₂)
xred = edges[idx]
yred = weights[idx]

p₃ = plot(xblue, yblue,
    ylims=(0,150),
    seriestype=:bar, colour=:darkblue, label = "N > 80",
    xlabel = "x [mm]", ylabel = "number of molecules",
    title = "velocity-jump process")
    plot!(xwhite, ywhite,
    seriestype=:bar, colour=:white, label = "80 >= N > 30")
    plot!(xred, yred,
    seriestype=:bar, colour=:red, label = "30 >= N")
    plot!(x, bound₁*ones(length(x)), linestyle=:dash, colour=:black, label=false)
    plot!(x, bound₂*ones(length(x)), linestyle=:dash, colour=:black, label=false)


layout = @layout [p₁ p₂ p₃]
ptotal = Plots.plot(p₁, p₂, p₃; layout = layout,  colorbar = false, size = (2400, 800),  m